<h1> Method 1: Custom XGB Container </h1>

This notebook demonstrates how to build and use a custom Docker container for training with Amazon SageMaker that leverages on the <strong>Script Mode</strong> execution that is implemented by the sagemaker-containers library. Reference documentation is available at https://github.com/aws/sagemaker-containers

We start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the custom Docker container and a default Amazon S3 bucket to be used by Amazon SageMaker.

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'script-mode-container-xgb'

ecr_repository_name = ecr_namespace + prefix
role = "arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154"
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

342474125894
ap-southeast-1
arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154
sagemaker-ap-southeast-1-342474125894


<h3>Build and push the container</h3>
We are now ready to build this container and push it to Amazon ECR. This task is executed using a shell script stored in the ../script/ folder. Let's take a look at this script and then execute it.

In [42]:
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

Sending build context to Docker daemon  13.31kB
Step 1/16 : FROM ubuntu:16.04
 ---> 13c9f1285025
Step 2/16 : LABEL maintainer="Giuseppe A. Porcelli"
 ---> Using cache
 ---> 6bbf3d07c68d
Step 3/16 : ARG PYTHON=python3
 ---> Using cache
 ---> 8e254b9ef0a0
Step 4/16 : ARG PYTHON_PIP=python3-pip
 ---> Using cache
 ---> 84c928b11bb3
Step 5/16 : ARG PIP=pip3
 ---> Using cache
 ---> 65e780b1f9d7
Step 6/16 : ARG PYTHON_VERSION=3.6.6
 ---> Using cache
 ---> 03bab72f170e
Step 7/16 : RUN apt-get update && apt-get install -y --no-install-recommends software-properties-common &&     add-apt-repository ppa:deadsnakes/ppa -y &&     apt-get update && apt-get install -y --no-install-recommends     build-essential     ca-certificates     curl     wget     git     libopencv-dev     openssh-client     openssh-server     vim     zlib1g-dev &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> e29c159657d9
Step 8/16 : RUN wget https://www.python.org/ftp/python/$PYTHON_VERSION/Python-$PYTHON_VERSION.tgz

<h3>Training with Amazon SageMaker</h3>

Once we have correctly pushed our container to Amazon ECR, we are ready to start training with Amazon SageMaker, which requires the ECR path to the Docker container used for training as parameter for starting a training job.

In [31]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

342474125894.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-training-containers/script-mode-container-xgb:latest


You can realize that the training code has been implemented as a standard Python script, that will be invoked by the sagemaker-containers library passing hyperparameters as arguments. This way of invoking training script is indeed called <strong>Script Mode</strong> for Amazon SageMaker containers.

<h3>Prepare Data</h3>

Now, we upload some dummy data to Amazon S3, in order to define our S3-based training channels.

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd

X, y = make_classification(
    n_samples=100, n_features=5, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=3
)
print(f"X: {X.shape}, y:{y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
train_df = pd.concat([y_train, X_train], axis=1)
train_df.columns = range(train_df.shape[1])
test_df = pd.concat([y_test, X_test], axis=1)
test_df.columns = range(test_df.shape[1])

print(f"train_df: {train_df.shape}, test_df:{test_df.shape}")

X: (100, 5), y:(100,)
train_df: (70, 6), test_df:(30, 6)


In [6]:
train_filename = "../test_data/train/train.csv"
test_filename = "../test_data/val/test.csv"
train_df.to_csv(train_filename, header=True, index=False)
test_df.to_csv(test_filename, header=True, index=False)

train_uri = sagemaker_session.upload_data(train_filename, bucket, prefix + '/train')
test_uri = sagemaker_session.upload_data(test_filename, bucket, prefix + '/val')
print(train_uri)
print(test_uri)
#! rm $train_filename $test_filename

s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb/train/train.csv
s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb/val/test.csv


<h3>Training

Finally, we can execute the training job by calling the fit() method of the generic Estimator object defined in the Amazon SageMaker Python SDK (https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/estimator.py). This corresponds to calling the CreateTrainingJob() API (https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html).

In [43]:
import sagemaker
import json

# JSON encode hyperparameters to avoid showing some info messages raised by the sagemaker-containers library.
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({
    "hp1": "value1",
    "hp2": 300,
    "hp3": 0.001})

est = sagemaker.estimator.Estimator(container_image_uri,
                                    role,
                                    train_instance_count=1, 
                                    #train_instance_type='local', # we use local mode
                                    train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix,
                                    hyperparameters=hyperparameters)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')
#print(train_config.config, val_config)
est.fit({'train': train_config, 'validation': val_config })

2020-08-10 16:33:05 Starting - Starting the training job...
2020-08-10 16:33:07 Starting - Launching requested ML instances......
2020-08-10 16:34:16 Starting - Preparing the instances for training...
2020-08-10 16:34:54 Downloading - Downloading input data
2020-08-10 16:34:54 Training - Downloading the training image.....2020-08-10 16:35:41,116 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-10 16:35:47,351 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-10 16:35:47,364 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-08-10 16:35:47,375 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    

<h3>check model artifact</h3>
    
make sure it is Booster type,

In [48]:
artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
print(artifact_path)


s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz


In [49]:
! aws s3 cp s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz .
! tar -xvf model.tar.gz
! rm model.tar.gz

download: s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz to ./model.tar.gz
x model.pth


In [50]:
import pickle
with open('model.pth', 'rb') as f:
    model = pickle.load(f)
    
model

<h1> Method 2: Prebuilt container </h1>

Prebuilt container has both sagemaker-container/sagemaker-inference install in a single image.


In [71]:
from sagemaker.xgboost.estimator import XGBoost

xgb_estimator = XGBoost(
    entry_point="train.py",
    source_dir="../docker/code",
    #hyperparameters=hyperparameters,
    role=role,
    train_instance_count=1,
    train_instance_type="ml.m5.2xlarge",
    framework_version="0.90-2",
)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')
#print(train_config.config, val_config)
xgb_estimator.fit({'train': train_config, 'validation': val_config })

2020-08-10 17:40:58 Starting - Starting the training job...
2020-08-10 17:41:01 Starting - Launching requested ML instances......
2020-08-10 17:42:28 Starting - Preparing the instances for training...
2020-08-10 17:42:59 Downloading - Downloading input data...
2020-08-10 17:43:17 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module train does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python -m pip install . 
Processing /opt/ml/code
  Created wheel for train: filename=train-1.0.0-py2.py3-none-any.whl size=8600 sha256=332aa74f90b2cd66aa551c15e162b8319837108c

In [72]:
xgb_estimator.deploy(initial_instance_count=1, instance_type="ml.t2.large")

---------------!

In [73]:
import boto3

runtime_client = boto3.client('runtime.sagemaker')


payload = "1,2,3,4,5"

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)

result = response['Body'].read().decode('ascii')
print(result)

[[0.33649730682373047, 0.33273303508758545, 0.33076968789100647]]


In [85]:
from sagemaker.xgboost.model import XGBoostPredictor

endpoint_name = 'sagemaker-xgboost-2020-08-10-17-41-03-430'
payload = "1,2,3,4,5"
xgb_predictor = XGBoostPredictor(endpoint_name, )


In [86]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])
predictions

text/csv
application/x-npy


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://ap-southeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-southeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-xgboost-2020-08-10-17-41-03-430 in account 342474125894 for more information.